# 🧪 Anomaly Scoring & Heatmaps

In this step, we transform ViT-based features into meaningful **anomaly scores** and **visual heatmaps**.

Using the patch-level embeddings from *test images*, we compare them against the **feature bank** built from `train/good` data. This allows us to detect both image-level anomalies and localize defective regions.

---

### Goals of this notebook:

- Compute **image-level anomaly scores** using:
  - **KNN distance** (via FAISS)
  - Optionally: **Mahalanobis distance**
- Generate **pixel-level heatmaps** by projecting patch distances to spatial grids
- Upsample and visualize heatmaps as overlays on the original images
- Determine an optimal threshold for binary decisions (Youden’s J or best F1)
- Evaluate detection performance using standard metrics (AUROC, PRO, etc.)

---

This scoring approach works **fully unsupervised**, relying only on normal (`train/good`) examples.

By the end of this notebook, you’ll be able to:
- Score all test images
- Visualize and interpret heatmaps
- Quantify anomaly detection performance


## 1️⃣ Load Features & Metadata

In this section, we load the precomputed ViT embeddings and metadata generated during the feature extraction step.

Specifically, we will:
- Load `.npz` feature files (CLS + patch embeddings) for each category
- Load the corresponding `.csv` metadata files
- Merge them into a single DataFrame for easy access and filtering

This unified structure will allow us to:
- Quickly isolate `train/good` features (for scoring)
- Select `test` images for evaluation
- Link embeddings back to image paths for visualization

---

Each entry contains:
- `cls`: global image embedding (ViT CLS token)
- `patches`: local patch embeddings
- `patch_hw`: patch grid shape (e.g., 16×16)
- metadata (path, label, split, etc.)


In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
import pickle

BASE_DIR = Path("..").resolve()

CFG = {
    "dino": {
        "FEAT_DIR": BASE_DIR / "features_dinov2_b14",
        "BANK_DIR": BASE_DIR / "featurebanks" / "dinov2_b14",
        "FEAT_NPZ_TPL": "{cat}_dinov2_vitb14.npz",
        "BANK_NPZ_TPL": "{cat}_bank_dinov2_b14.npz",
    },
    "mae": {
        "FEAT_DIR": BASE_DIR / "features_mae_b16",
        "BANK_DIR": BASE_DIR / "featurebanks" / "mae_b16",
        "FEAT_NPZ_TPL": "{cat}_mae_b16.npz",
        "BANK_NPZ_TPL": "{cat}_bank_mae_b16.npz",
    },
}

def categories_from_feat_dir(feat_dir: Path):
    cats = []
    for p in feat_dir.glob("*_meta.csv"):
        name = p.name
        if name.endswith("_meta.csv"):
            cats.append(name[:-len("_meta.csv")])
    return sorted(set(cats))

def load_feature_data(feat_dir: Path, feat_tpl: str, category: str):
    feat_file = feat_dir / feat_tpl.format(cat=category)
    meta_file = feat_dir / f"{category}_meta.csv"
    data = np.load(feat_file)
    df = pd.read_csv(meta_file)
    return {
        "patches": data["patches"],
        "cls": data.get("cls"),
        "patch_hw": tuple(data["patch_hw"]),
        "meta": df
    }

def load_feature_bank(bank_dir: Path, bank_tpl: str, category: str):
    bank_file = bank_dir / bank_tpl.format(cat=category)
    meta_file = bank_dir / f"{category}_bank_meta.csv"
    data = np.load(bank_file)
    df = pd.read_csv(meta_file)
    return {
        "patches": data["patches"],
        "patch_hw": tuple(data["patch_hw"]),
        "meta": df
    }

def save_dicts_to_disk(backbone: str, features_all: dict, banks: dict, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    for cat in features_all:
        with open(out_dir / f"{cat}_features.pkl", "wb") as f:
            pickle.dump(features_all[cat], f)
        with open(out_dir / f"{cat}_bank.pkl", "wb") as f:
            pickle.dump(banks[cat], f)
        print(f"✅ Saved {cat} ({backbone}) to {out_dir}")

def load_backbone(name: str):
    cfg = CFG[name]
    feat_dir, bank_dir = cfg["FEAT_DIR"], cfg["BANK_DIR"]
    feat_tpl, bank_tpl = cfg["FEAT_NPZ_TPL"], cfg["BANK_NPZ_TPL"]

    categories = categories_from_feat_dir(feat_dir)
    features_all, banks = {}, {}
    for cat in categories:
        features_all[cat] = load_feature_data(feat_dir, feat_tpl, cat)
        banks[cat] = load_feature_bank(bank_dir, bank_tpl, cat)
    return categories, features_all, banks

# Load and save DINO
cats_dino, features_all_dino, banks_dino = load_backbone("dino")
save_dicts_to_disk("dino", features_all_dino, banks_dino, BASE_DIR / "cached_dicts" / "dino")

# Load and save MAE
cats_mae, features_all_mae, banks_mae = load_backbone("mae")
save_dicts_to_disk("mae", features_all_mae, banks_mae, BASE_DIR / "cached_dicts" / "mae")




✅ Saved bottle (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved cable (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved capsule (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved carpet (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved grid (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved hazelnut (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved leather (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved metal_nut (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved pill (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved screw (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved tile (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved toothbrush (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved transistor (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved wood (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved zipper (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved bottle (mae) to C:\Users\Fr

### Data Structures

After loading, we store the results in two dictionaries.  
This design separates **all extracted embeddings** from the **reference banks**,  
making evaluation, visualization, and anomaly scoring more convenient.

---

- **`features_all`**  
  Contains the **full embeddings** (train + test) for each category.  
  Each entry is a dictionary with:
  - `cls`: global image embeddings `[N, D]`
  - `patches`: patch embeddings `[N, P, D]`
  - `patch_hw`: patch grid dimensions `(H, W)`
  - `meta`: metadata DataFrame (paths, labels, splits)

  → Used when we need to compare across *all images* (e.g., evaluation, visualization).

---

- **`banks`**  
  Contains only the **feature bank**: patch embeddings from `train/good` images.  
  Each entry is a dictionary with:
  - `patches`: patch embeddings `[N, P, D]`
  - `patch_hw`: patch grid dimensions `(H, W)`
  - `meta`: metadata DataFrame for good samples

  → Used as the **reference set** for anomaly scoring: test patches are compared against this bank.


These structures allow us to seamlessly switch between global evaluation (`features_all`) and reference-based scoring (`banks`). In the next step, we leverage them to compute image-level anomaly scores and pixel-level heatmaps.



## 2️⃣ Build Search Index & Image-Level Scoring

After extracting features, the next step is to organize the  
**reference distribution of normal data** and to perform  
**image-level anomaly scoring** based on this reference.  
This combined stage forms the *core* of the anomaly detection pipeline.

Two complementary approaches are implemented:

- **KNN-based feature banks (non-parametric):**  
  Patch embeddings from `train/good` images are stored in a  
  nearest-neighbor index (FAISS, cosine similarity with PCA to 128D).  
  → At inference, each test patch is compared to its closest neighbors  
  in this bank.

- **Mahalanobis distribution (parametric):**  
  For each category, the mean vector (μ) and a shrunk covariance matrix (Σ̂)  
  of `train/good` patches are estimated.  
  → Anomaly scores are then derived from the squared Mahalanobis distance,  
  measuring how well test patches fit into the global distribution.

---

### Image-Level Scoring

For every test image:

1. **Patch-level comparison:**  
   Each patch embedding is compared against the chosen reference  
   (nearest neighbors for KNN, multivariate distance for Mahalanobis).  

2. **Patch scoring:**  
   Each patch receives an anomaly score (distance).  

3. **Image aggregation:**  
   The **Top-K most anomalous patches** are averaged, yielding  
   a single **image-level anomaly score**.  

---

By uniting the **construction of the normal reference** with the  
**aggregation of patch scores into image-level scores**,  
this stage transforms raw features into **directly usable anomaly scores**.  
These scores provide the basis for evaluation, threshold selection,  
and visualization in later steps.




### 🔹2.1 KNN Implementation (with FAISS) 

In our pipeline, the **KNN-based anomaly scoring** is implemented as a 
streaming procedure to stay RAM-safe while handling large feature banks.  
The core logic is encapsulated in [`src/scoring_knn.py`](src/scoring_knn.py).

**Workflow:**
1. **Feature Bank Creation:**  
   All patch embeddings from `train/good` images are stored as `.pkl` files 
   (one `*_features.pkl` for metadata + test/train splits, one `*_bank.pkl` 
   containing the reference patches).

2. **Streaming per Category:**  
   Instead of loading the entire dataset into memory, 
   `score_backbone_streaming()` processes one category at a time.  
   For each category:
   - Load the feature bank (`train/good` patches).  
   - Apply **PCA reduction to 128 dimensions** (from the original 768D).  
     → This lowers memory usage and speeds up FAISS queries, while 
       retaining most discriminative information.  
   - Build a **FAISS index** with **cosine similarity** as metric.  

3. **Patch-Level k-NN Search:**  
   Each test patch is queried against the index.  
   - We use **cosine similarity** instead of L2 distance, because 
     embeddings from pre-trained ViTs (DINO/MAE) are generally 
     normalized and directional information is more robust for 
     anomaly detection.  
   - Anomaly distance = `1 - cosine_similarity`.  
   - For each patch, the **mean distance of its k nearest neighbors** is computed.  

4. **Image-Level Aggregation:**  
   From the patch-level scores, the top-K most anomalous patches are 
   averaged → this yields the **final image-level anomaly score**.  

5. **Output:**  
   Results are appended into a CSV file, one row per test image:  

   | idx | path | category | raw_label | label | image_score |  
   |-----|------|----------|-----------|-------|-------------|  

**Advantages:**
- Memory-efficient (streaming, batch search).  
- PCA (128D) greatly reduces RAM and compute requirements.  
- Cosine similarity aligns better with the embedding geometry of 
  transformers, leading to more stable scoring.  
- GPU-accelerated nearest neighbor queries via FAISS.  

This implementation ensures that anomaly scoring remains **scalable and efficient**, 
while preserving the accuracy benefits of non-parametric k-NN methods.

---




In [39]:
import pandas as pd

# CSV laden, zur Sicherheit Separator explizit angeben
df = pd.read_csv("../scores_knn/scores_STREAM_dino_cosine_k5_top5.csv", sep=",")

# image_score in float konvertieren
df["image_score"] = pd.to_numeric(df["image_score"], errors="coerce")

# Kurzer Check
print(df.dtypes)
print(df["image_score"].head())

# Statistical summary
display(df.describe()[["image_score"]])

# Nur "good" und "defect" berücksichtigen
mean_scores = df[df["label"].isin(["good", "defect"])] \
    .groupby("label")["image_score"].mean()

print("\n🔍 Mean anomaly score per label (cleaned):")
print(mean_scores)

if {"good", "defect"} <= set(mean_scores.index):
    delta = mean_scores["defect"] - mean_scores["good"]
    print(f"\n📈 Difference (defect - good): {delta:.4f}")




idx              int64
path            object
category        object
raw_label       object
label           object
image_score    float64
dtype: object
0    0.357246
1    0.343680
2    0.337821
3    0.422264
4    0.369904
Name: image_score, dtype: float64


,image_score
count,1725.000000
mean,0.177325
std,0.093315
min,0.036036
25%,0.095347
50%,0.164082
75%,0.237266
max,0.641549



🔍 Mean anomaly score per label (cleaned):
label
defect    0.207137
good      0.097017
Name: image_score, dtype: float64

📈 Difference (defect - good): 0.1101


In [40]:
import pandas as pd
from pathlib import Path

# --- Paths ---
p_dino = Path("../scores_knn/scores_STREAM_dino_cosine_k5_top5.csv")
p_mae  = Path("../scores_knn/scores_STREAM_mae_cosine_k5_top5.csv")

# --- Load + clean helper ---
def load_clean_scores(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Ensure numeric (in case it's stored as string)
    if df["image_score"].dtype != "float":
        df["image_score"] = pd.to_numeric(df["image_score"], errors="coerce")
    # Keep only valid labels
    df = df[df["label"].isin(["good", "defect"])].copy()
    # Drop corrupted rows if any
    df = df.dropna(subset=["image_score", "label"])
    return df

df_dino = load_clean_scores(p_dino)
df_mae  = load_clean_scores(p_mae)

# --- Mean per label ---
mean_dino = df_dino.groupby("label")["image_score"].mean()
mean_mae  = df_mae.groupby("label")["image_score"].mean()

# Align index to ensure consistent order
label_order = ["good", "defect"]
mean_dino = mean_dino.reindex(label_order)
mean_mae  = mean_mae.reindex(label_order)

# --- Merge + deltas ---
mean_df = pd.DataFrame({
    "DINO (KNN)": mean_dino,
    "MAE (KNN)":  mean_mae
})
mean_df["Δ (defect - good) DINO"] = mean_dino.loc["defect"] - mean_dino.loc["good"]
mean_df["Δ (defect - good) MAE"]  = mean_mae.loc["defect"]  - mean_mae.loc["good"]

print("🔍 Mean anomaly score per label (KNN):")
display(mean_df)

# Compact delta-only output
delta = pd.Series({
    "DINO (KNN)": mean_dino.loc["defect"] - mean_dino.loc["good"],
    "MAE (KNN)":  mean_mae.loc["defect"]  - mean_mae.loc["good"],
}, name="Δ (defect - good)")
print("\n📈 Score difference (defect - good):")
display(delta)

# --- Optional: per-category check (uncomment display to view) ---
per_cat = (
    pd.concat(
        [df_dino.assign(model="DINO"), df_mae.assign(model="MAE")],
        ignore_index=True
    )
    .groupby(["model", "category", "label"])["image_score"].mean()
    .unstack("label")
    .assign(delta=lambda x: x["defect"] - x["good"])
    .sort_values(["model", "delta"], ascending=[True, False])
)
# display(per_cat)  # Uncomment if you want to inspect per-category deltas



🔍 Mean anomaly score per label (KNN):


,DINO (KNN),MAE (KNN),Δ (defect - good) DINO,Δ (defect - good) MAE
label,,,,
good,0.097017,0.148898,0.11012,0.034547
defect,0.207137,0.183444,0.11012,0.034547



📈 Score difference (defect - good):


DINO (KNN)    0.110120
MAE (KNN)     0.034547
Name: Δ (defect - good), dtype: float64


#### 📊 Observed Score Distribution (DINO, cosine, PCA=128D, k=5, TopK=5)

*The following results were obtained by running the pipeline in **Google Colab**.*

**Global stats (all test images, N=1,725):**
- count: **1725**
- mean: **0.1773**
- std: **0.0933**
- min / 25% / 50% / 75% / max: **0.0361 / 0.0953 / 0.1641 / 0.2373 / 0.6416**

**By label:**
- good → mean **0.0970**
- defect → mean **0.2071**

**Interpretation:**
- Scores are clearly **higher for defective images** (≈2× on average), which indicates good separability.
- The overall spread (up to ~0.64) shows that strongly anomalous cases stand out clearly.
- A practical starting **threshold** typically falls between **0.12–0.18**  
  (more precisely determined via ROC; e.g., Youden’s J = argmax(tpr − fpr)).

**Implications for downstream evaluation:**
- A **high AUROC** (often ≥0.9) can be expected given this mean gap.  
- For deployment: thresholds should be calibrated per category (e.g., ROC-based or using the 95th percentile of *good* scores).


### 🔹 2.2 Mahalanobis Implementation

In addition to the non-parametric k-NN approach, we also evaluate a **parametric anomaly scoring method** based on the **Mahalanobis distance**.  
Here, the assumption is that the embeddings of *normal* patches (from `train/good`) approximately follow a **multivariate Gaussian distribution**.

**Workflow:**
1. **Fit Distribution:**  
   - Collect all patch embeddings from `train/good`.  
   - Compute the **mean vector** (μ) and a **shrunk covariance matrix** (Σ̂).  
   - Shrinkage (convex combination with identity) ensures stable inversion even when samples are fewer than dimensions or features are correlated.  

2. **O PCA (128D):**  
   To reduce dimensionality and improve numerical stability, PCA is trained on a sample of patches and applied to both train and test embeddings.  

3. **Patch-Level Scoring:**  
   Each test patch is assigned a score equal to its **squared Mahalanobis distance**:  
   $d^2(x) = (x - \mu)^T \hat{\Sigma}^{-1} (x - \mu)$


4. **Image-Level Aggregation:**  
   For each test image, the **Top-K highest patch scores** are averaged, yielding the final **image-level anomaly score**.  

**Advantages:**
- Parametric: explicitly models the distribution of normal data.  
- More compact than storing all patches (no large FAISS index required).  
- Shrinkage + PCA improve stability and reduce overfitting.  

This method provides a **complementary perspective** to k-NN: instead of comparing to individual neighbors, it measures how well a sample fits into the global distribution of normal features.


In [38]:
import gc
import time
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from sklearn.decomposition import PCA

# ----------------- PCA + Mahalanobis Core ----------------- #
def train_pca_on_sample(cache_dir, backbone, categories, pca_dim, sample_cap=200_000, whitening_power=0.0):
    if pca_dim is None:
        return None
    print(f"⏳ PCA: {backbone}, dim={pca_dim}")
    sample_cat = categories[0]
    with open(cache_dir / backbone / f"{sample_cat}_bank.pkl", "rb") as f:
        bank = pickle.load(f)
    sb = bank["patches"].reshape(-1, bank["patches"].shape[-1]).astype(np.float32)
    idx = np.random.choice(len(sb), size=min(sample_cap, len(sb)), replace=False)
    whiten = whitening_power < 0
    pca = PCA(n_components=pca_dim, whiten=whiten)
    pca.fit(sb[idx])
    return pca

def apply_pca_inplace(x: np.ndarray, pca: Optional[PCA]) -> np.ndarray:
    if pca is None:
        return np.ascontiguousarray(x.astype(np.float32, copy=False))
    y = pca.transform(x)
    return np.ascontiguousarray(y.astype(np.float32, copy=False))

def fit_mean_cov_shrunk(X: np.ndarray, shrinkage_alpha: float = 0.1):
    mu = X.mean(axis=0)
    Xc = X - mu
    S = np.cov(Xc, rowvar=False, ddof=1).astype(np.float64)
    trace = np.trace(S) if np.isfinite(np.trace(S)) else np.sum(np.var(X, axis=0))
    lam = trace / max(S.shape[0], 1)
    S_hat = (1.0 - shrinkage_alpha) * S + shrinkage_alpha * lam * np.eye(S.shape[0])
    return mu.astype(np.float32), S_hat.astype(np.float32)

def invert_posdef(S: np.ndarray) -> np.ndarray:
    try:
        L = np.linalg.cholesky(S)
        Linv = np.linalg.solve(L, np.eye(L.shape[0]))
        return (Linv.T @ Linv).astype(np.float32)
    except np.linalg.LinAlgError:
        return np.linalg.pinv(S, rcond=1e-6).astype(np.float32)

def mahalanobis_sq_batch(X: np.ndarray, mu: np.ndarray, S_inv: np.ndarray, batch_size: int = 50_000):
    out = np.empty(X.shape[0], dtype=np.float32)
    for s in range(0, X.shape[0], batch_size):
        e = min(s + batch_size, X.shape[0])
        Xc = X[s:e] - mu
        out[s:e] = np.einsum("ij,jk,ik->i", Xc, S_inv, Xc, optimize=True)
    return out


In [47]:
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import time, gc, pickle
import numpy as np
import pandas as pd

# --- helpers ---
def _infer_hw(P: int) -> Tuple[int, int]:
    r = int(np.sqrt(P))
    return (r, r) if r * r == P else (1, P)

def _save_path_for_npz(patch_out_root: Path, backbone: str, meta_row) -> Path:
    """
    Mirror dataset structure to avoid filename collisions:
    <root>/<backbone>/<category>/<split>/<raw_label>/<filename>.png.npz
    """
    p = Path(str(meta_row["path"]))
    cat   = str(meta_row["category"])
    split = str(meta_row["split"]).lower()
    raw   = str(meta_row["raw_label"])
    fname = p.name  # e.g. "000.png"
    save_dir = patch_out_root / backbone / cat / split / raw
    save_dir.mkdir(parents=True, exist_ok=True)
    return save_dir / f"{fname}.npz"

# --- main ---
def score_all_backbones_mahalanobis_separate(
    cache_dir: Path,
    out_files: Dict[str, Path],             # backbone -> CSV path
    backbones: List[str],                   # e.g. ["dino", "mae"]
    k_top_patches: int = 5,
    pca_dim: Optional[int] = 128,
    pca_whitening_power: float = 0.0,
    shrinkage_alpha: float = 0.1,
    query_batch_size: int = 20_000,
    save_patch_scores: bool = True,
    patch_out_root: Optional[Path] = None,
):
    for backbone in backbones:
        out_file = out_files[backbone]
        out_file.parent.mkdir(parents=True, exist_ok=True)
        wrote_header = False

        b_dir = cache_dir / backbone
        categories = sorted(p.stem.replace("_features", "") for p in b_dir.glob("*_features.pkl"))
        print(f"\n== [{backbone.upper()}] {len(categories)} categories ==")

        # Train PCA on first category (on the patch bank)
        pca = train_pca_on_sample(cache_dir, backbone, categories, pca_dim, whitening_power=pca_whitening_power)

        for cat in categories:
            t0 = time.time()

            # Load features and bank
            with open(b_dir / f"{cat}_features.pkl", "rb") as f: feats = pickle.load(f)
            with open(b_dir / f"{cat}_bank.pkl", "rb") as f: bank = pickle.load(f)

            # Fit distribution (mean & shrunk covariance)
            bank_mat = bank["patches"].reshape(-1, bank["patches"].shape[-1])
            bank_mat = apply_pca_inplace(bank_mat, pca)
            mu, S_hat = fit_mean_cov_shrunk(bank_mat, shrinkage_alpha=shrinkage_alpha)
            S_inv = invert_posdef(S_hat)

            # Extract test patch features
            patches = feats["patches"].astype(np.float32)  # [N, P, D]
            meta = feats["meta"]
            is_test = meta["split"].astype(str).str.lower().eq("test").values
            test_idx = np.where(is_test)[0]
            P, D = patches.shape[1], patches.shape[2]
            X = patches[is_test].reshape(-1, D)
            X = apply_pca_inplace(X, pca)

            # Mahalanobis distances per patch
            patch_scores = mahalanobis_sq_batch(X, mu, S_inv, batch_size=query_batch_size).reshape(-1, P)

            # Top-K aggregation per image
            tk = min(k_top_patches, P)
            order = np.argsort(patch_scores, axis=1)
            topk_idx = order[:, -tk:]
            img_scores = patch_scores[np.arange(len(patch_scores))[:, None], topk_idx].mean(axis=1)

            # Save per-image patch scores (UNIQUE paths)
            if save_patch_scores and patch_out_root is not None:
                H, W = _infer_hw(P)
                for j, i in enumerate(test_idx):
                    save_path = _save_path_for_npz(patch_out_root, backbone, meta.iloc[i])
                    np.savez_compressed(
                        save_path,
                        patch_scores=patch_scores[j],                # (P,)
                        image_score=float(img_scores[j]),           # scalar
                        grid_hw=np.array([H, W], dtype=np.int32),   # e.g., [16, 16]
                        topk_idx=topk_idx[j].astype(np.int32),
                        meta=dict(
                            path=str(meta.iloc[i]["path"]),
                            category=str(meta.iloc[i]["category"]),
                            split=str(meta.iloc[i]["split"]),
                            raw_label=str(meta.iloc[i]["raw_label"]),
                            label=str(meta.iloc[i]["label"]),
                            backbone=str(backbone),
                        ),
                    )

            # Append CSV
            rows = []
            for j, i in enumerate(test_idx):
                rows.append({
                    "idx": int(i),
                    "category": meta.iloc[i]["category"],
                    "path": meta.iloc[i]["path"],
                    "raw_label": meta.iloc[i]["raw_label"],
                    "label": meta.iloc[i]["label"],
                    "image_score": float(img_scores[j]),
                })
            df = pd.DataFrame(rows)
            df.to_csv(out_file, mode="a", header=not wrote_header, index=False)
            wrote_header = True

            # Cleanup
            del feats, bank, bank_mat, mu, S_hat, S_inv, patches, X, patch_scores, img_scores, df
            gc.collect()

            print(f"• {backbone}/{cat}: {len(rows)} imgs in {time.time() - t0:.1f}s")

        print(f"📄 Done → {out_file}")




In [48]:
from pathlib import Path

# Shared configuration
CACHE_DIR = Path("../cached_dicts")
PATCH_OUT_ROOT = Path("../scores_mahalanobis/patch_scores")
PCA_DIM = 128
K_TOP_PATCHES = 5
BATCH_SIZE = 20_000
SHRINKAGE = 0.1
WHITENING = 0.0

# --- Run 1: DINO ---
score_all_backbones_mahalanobis_separate(
    cache_dir=CACHE_DIR,
    out_files={
        "dino": Path("../scores_mahalanobis/scores_mahalanobis_dino.csv")
    },
    backbones=["dino"],
    k_top_patches=K_TOP_PATCHES,
    pca_dim=PCA_DIM,
    pca_whitening_power=WHITENING,
    shrinkage_alpha=SHRINKAGE,
    query_batch_size=BATCH_SIZE,
    save_patch_scores=True,
    patch_out_root=PATCH_OUT_ROOT,
)

# --- Run 2: MAE ---
score_all_backbones_mahalanobis_separate(
    cache_dir=CACHE_DIR,
    out_files={
        "mae": Path("../scores_mahalanobis/scores_mahalanobis_mae.csv")
    },
    backbones=["mae"],
    k_top_patches=K_TOP_PATCHES,
    pca_dim=PCA_DIM,
    pca_whitening_power=WHITENING,
    shrinkage_alpha=SHRINKAGE,
    query_batch_size=BATCH_SIZE,
    save_patch_scores=True,
    patch_out_root=PATCH_OUT_ROOT,
)




== [DINO] 15 categories ==
⏳ PCA: dino, dim=128
• dino/bottle: 83 imgs in 0.4s
• dino/cable: 150 imgs in 0.6s
• dino/capsule: 132 imgs in 0.5s
• dino/carpet: 117 imgs in 0.6s
• dino/grid: 78 imgs in 0.5s
• dino/hazelnut: 110 imgs in 0.7s
• dino/leather: 124 imgs in 0.6s
• dino/metal_nut: 115 imgs in 0.5s
• dino/pill: 167 imgs in 0.7s
• dino/screw: 160 imgs in 0.7s
• dino/tile: 117 imgs in 0.5s
• dino/toothbrush: 42 imgs in 0.2s
• dino/transistor: 100 imgs in 0.5s
• dino/wood: 79 imgs in 0.5s
• dino/zipper: 151 imgs in 0.6s
📄 Done → ..\scores_mahalanobis\scores_mahalanobis_dino.csv

== [MAE] 15 categories ==
⏳ PCA: mae, dim=128
• mae/bottle: 83 imgs in 0.4s
• mae/cable: 150 imgs in 0.6s
• mae/capsule: 132 imgs in 0.6s
• mae/carpet: 117 imgs in 0.6s
• mae/grid: 78 imgs in 0.5s
• mae/hazelnut: 110 imgs in 0.7s
• mae/leather: 124 imgs in 0.6s
• mae/metal_nut: 115 imgs in 0.5s
• mae/pill: 167 imgs in 0.7s
• mae/screw: 160 imgs in 0.7s
• mae/tile: 117 imgs in 0.5s
• mae/toothbrush: 42 imgs 

In [33]:
import pandas as pd
from pathlib import Path

# --- Paths ---
p_dino = Path("../scores_mahalanobis/scores_mahalanobis_dino.csv")
p_mae  = Path("../scores_mahalanobis/scores_mahalanobis_mae.csv")

def analyze_scores(path: Path, model_name: str):
    print(f"\n=== {model_name} :: Mahalanobis scoring ===")
    
    # Load CSV
    df = pd.read_csv(path, sep=",")
    
    # Ensure numeric
    df["image_score"] = pd.to_numeric(df["image_score"], errors="coerce")
    
    # Quick type / head check
    print(df.dtypes)
    print(df["image_score"].head())
    
    # Statistical summary
    display(df.describe()[["image_score"]])
    
    # Keep only good/defect
    mean_scores = (
        df[df["label"].isin(["good", "defect"])]
        .groupby("label")["image_score"]
        .mean()
    )
    
    print("\n🔍 Mean anomaly score per label (cleaned):")
    print(mean_scores)
    
    # Difference defect - good
    if {"good", "defect"} <= set(mean_scores.index):
        delta = mean_scores["defect"] - mean_scores["good"]
        print(f"\n📈 Difference (defect - good): {delta:.4f}")

# --- Run for both backbones ---
analyze_scores(p_dino, "DINO")
analyze_scores(p_mae,  "MAE")



=== DINO :: Mahalanobis scoring ===
idx             object
category        object
path            object
raw_label       object
label           object
image_score    float64
dtype: object
0    320.992706
1    297.331390
2    308.148987
3    315.113953
4    330.248535
Name: image_score, dtype: float64


,image_score
count,6921.000000
mean,418.711800
std,214.234865
min,157.835602
25%,260.797150
50%,338.385559
75%,508.260559
max,1473.911621



🔍 Mean anomaly score per label (cleaned):
label
defect    482.330091
good      245.735683
Name: image_score, dtype: float64

📈 Difference (defect - good): 236.5944

=== MAE :: Mahalanobis scoring ===
idx             object
category        object
path            object
raw_label       object
label           object
image_score    float64
dtype: object
0    212.623291
1    195.192902
2    240.066895
3    199.570724
4    217.294144
Name: image_score, dtype: float64


,image_score
count,5175.000000
mean,310.277570
std,294.352502
min,67.903641
25%,169.548019
50%,231.408295
75%,353.813065
max,3728.071045



🔍 Mean anomaly score per label (cleaned):
label
defect    338.13229
good      235.24280
Name: image_score, dtype: float64

📈 Difference (defect - good): 102.8895


In [37]:
import pandas as pd
from pathlib import Path

# --- Paths ---
p_dino = Path("../scores_mahalanobis/scores_mahalanobis_dino.csv")
p_mae  = Path("../scores_mahalanobis/scores_mahalanobis_mae.csv")

# --- Load + clean helper ---
def load_clean_scores(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Ensure numeric (in case it's stored as string)
    if df["image_score"].dtype != "float":
        df["image_score"] = pd.to_numeric(df["image_score"], errors="coerce")
    # Keep only valid labels
    df = df[df["label"].isin(["good", "defect"])].copy()
    # Drop corrupted rows if any
    df = df.dropna(subset=["image_score", "label"])
    return df

df_dino = load_clean_scores(p_dino)
df_mae  = load_clean_scores(p_mae)

# --- Mean per label ---
mean_dino = df_dino.groupby("label")["image_score"].mean()
mean_mae  = df_mae.groupby("label")["image_score"].mean()

# Align index to ensure consistent order
label_order = ["good", "defect"]
mean_dino = mean_dino.reindex(label_order)
mean_mae  = mean_mae.reindex(label_order)

# --- Merge + deltas ---
mean_df = pd.DataFrame({
    "DINO (Mahalanobis)": mean_dino,
    "MAE (Mahalanobis)":  mean_mae
})
mean_df["Δ (defect - good) DINO"] = mean_dino.loc["defect"] - mean_dino.loc["good"]
mean_df["Δ (defect - good) MAE"]  = mean_mae.loc["defect"]  - mean_mae.loc["good"]

print("🔍 Mean anomaly score per label (Mahalanobis):")
display(mean_df)

# Compact delta-only output
delta = pd.Series({
    "DINO (Mahalanobis)": mean_dino.loc["defect"] - mean_dino.loc["good"],
    "MAE (Mahalanobis)":  mean_mae.loc["defect"]  - mean_mae.loc["good"],
}, name="Δ (defect - good)")
print("\n📈 Score difference (defect - good):")
display(delta)

# --- Optional: per-category check (uncomment display to view) ---
per_cat = (
    pd.concat(
        [df_dino.assign(model="DINO"), df_mae.assign(model="MAE")],
        ignore_index=True
    )
    .groupby(["model", "category", "label"])["image_score"].mean()
    .unstack("label")
    .assign(delta=lambda x: x["defect"] - x["good"])
    .sort_values(["model", "delta"], ascending=[True, False])
)
# display(per_cat)  # Uncomment if you want to inspect per-category deltas


🔍 Mean anomaly score per label (Mahalanobis):


,DINO (Mahalanobis),MAE (Mahalanobis),Δ (defect - good) DINO,Δ (defect - good) MAE
label,,,,
good,246.607456,235.242988,233.319498,102.889107
defect,479.926954,338.132095,233.319498,102.889107



📈 Score difference (defect - good):


DINO (Mahalanobis)    233.319498
MAE (Mahalanobis)     102.889107
Name: Δ (defect - good), dtype: float64

### 🔎 Summary of Results (KNN vs. Mahalanobis)

We evaluated anomaly scores using two backbones (**DINO** and **MAE**) with two distance metrics (**KNN** and **Mahalanobis**).  
The table below shows the mean anomaly score for *good* and *defect* samples, as well as the difference between them (defect – good).  
A larger difference indicates a clearer separation between normal and anomalous samples.

| Metric         | Model | Good (↓ better) | Defect (↑ better) | Δ (defect – good) |
|----------------|-------|-----------------|-------------------|-------------------|
| **KNN**        | DINO  | 0.097           | 0.207             | **0.110** |
|                | MAE   | 0.149           | 0.183             | 0.035 |
| **Mahalanobis**| DINO  | 246.6           | 479.9             | **233.3** |
|                | MAE   | 235.2           | 338.1             | 102.9 |

#### Key observations
- **DINO consistently outperforms MAE** in both metrics, achieving a larger separation (Δ) between good and defect.
- **Mahalanobis yields higher absolute score ranges** than KNN, but the relative separation is what matters — DINO again provides the clearest gap.
- **MAE shows weaker separation**, especially with KNN, which may limit its discriminative power for anomaly detection.

#### Next step: Heatmaps
While global scores provide evidence that DINO (especially with Mahalanobis) is more discriminative, they do not reveal *where* anomalies occur.  
The next step is **pixel-level heatmap generation**:
- Map patch-level distances back to the image grid (e.g., 16×16 feature map).
- Upsample to the original resolution.
- Overlay as heatmaps to visualize **localized defects**.

This will allow a **qualitative evaluation** and demonstrate whether the quantitative advantage of DINO also translates into clearer anomaly localization.



## 4️⃣ Heatmap Generation (Pixel-Level)

While image-level anomaly scores provide a single value per image (indicating whether it is normal or defective), they do not reveal **where** the anomaly occurs.  
To gain deeper insights, we generate **pixel-level heatmaps** that localize potential defects within the image.

The core idea is straightforward:

1. **Patch-level distances**: Each test image is split into patches (e.g., 16×16 grid for a 256×256 image). Distances between these patches and the normal feature distribution serve as local anomaly scores.
2. **Interpolation**: The patch-level grid is upsampled (e.g., bilinear interpolation) to match the original image resolution.
3. **Visualization**: The resulting map can be rendered as a heatmap or overlayed on the original image, clearly highlighting defective regions.

This step transforms anomaly detection from a **binary decision** into an interpretable, **explainable localization task** — essential for real-world quality inspection scenarios.


In [45]:

import pandas as pd
from pathlib import Path

CSV_FILE = Path("../scores_mahalanobis/scores_mahalanobis_dino.csv")
PATCH_DIR = Path("../scores_mahalanobis/patch_scores/dino")

# Load CSV
df = pd.read_csv(CSV_FILE)

summary = []
for cat, group in df.groupby("category"):
    csv_count = len(group)
    npz_count = len(list((PATCH_DIR / cat).glob("*.npz")))
    summary.append((cat, csv_count, npz_count, csv_count == npz_count))

# Print results
print(f"{'Category':12s} | {'CSV rows':8s} | {'NPZ files':9s} | Match?")
print("-"*45)
for cat, csv_c, npz_c, ok in summary:
    print(f"{cat:12s} | {csv_c:8d} | {npz_c:9d} | {ok}")


Category     | CSV rows | NPZ files | Match?
---------------------------------------------
bottle       |       83 |        22 | False
cable        |      150 |        58 | False
capsule      |      132 |        23 | False
carpet       |      117 |        28 | False
grid         |       78 |        21 | False
hazelnut     |      110 |        40 | False
leather      |      124 |        32 | False
metal_nut    |      115 |        25 | False
pill         |      167 |        26 | False
screw        |      160 |        41 | False
tile         |      117 |        33 | False
toothbrush   |       42 |        30 | False
transistor   |      100 |        60 | False
wood         |       79 |        21 | False
zipper       |      151 |        32 | False
